The superconducting magnet shows hysteresis for performing in higher fields (e.g. 200mT). Thus, going straight to zero field, a residual field of several mT (e.g. 6.5mT) remains. This code helps to clean up and bring back the system to a close to zero field condition without warming up the  magnet above its critical temperature.

In [1]:
print('Kernel alive')

Kernel alive


In [4]:
# Current position ={'rho': 0.069, 'theta': 58.9011, 'phi': 177.0225} 

In [5]:
from tqdm import tqdm
import time

def ramp_along_set_direction(sweep_steps=10, theta=None, phi=None, rho=None):
    # !!! Rho is in T !!!
    #First define the steps to ramp down the magnet
    # DEFINING MEASUREMENT PARAMETERS
    # DEFINING MAGNETIC FIELD VECTORS
    curr_pos_dict = magnetlogic.get_pos() #get current position
    print('Current position', curr_pos_dict,"\n\n")
    if not theta:
        theta = np.deg2rad(curr_pos_dict['theta'])
    if not phi:
        phi = np.deg2rad(curr_pos_dict['phi'])
    if not rho:
        rho = curr_pos_dict['rho']
    start = rho

    ###### Comment out for using current values
    # theta = np.deg2rad(54.87516187351408)
    # phi = np.deg2rad(178.5983892617904)
    # start = 0.2699180245926529

    stop = 0.01*rho
    sweep_arr = np.zeros(sweep_steps)
    sweep_arr[:-1] = np.logspace(np.log10(start), np.log10(stop), num=sweep_steps-1)
    aps_magnet.check_precision = 0.00001
    magnetlogic._checktime = 1.5

    b_arr = []
    for idx, rho_val in enumerate(sweep_arr):
        theta = np.pi-theta
        phi = np.pi+phi
        phi = phi if phi<2*np.pi else phi-2*np.pi
        move_dict =  {'rho':0, 'theta':np.rad2deg(theta), 'phi': np.rad2deg(phi)}
        print(move_dict)
        b_arr.append(move_dict)
        move_dict =  {'rho':rho_val, 'theta':np.rad2deg(theta), 'phi': np.rad2deg(phi)}
        print(move_dict)
        b_arr.append(move_dict)
    return b_arr

def ramp_along_axes(sweep_steps=10, axis="z", rho=None):
    # !!! Rho is in T !!!
    #First define the steps to ramp down the magnet
    # DEFINING MEASUREMENT PARAMETERS
    # DEFINING MAGNETIC FIELD VECTORS
    curr_pos_dict = magnetlogic.get_pos() #get current position
    print('Current position', curr_pos_dict,"\n\n")
    if axis is "x":
        theta = np.deg2rad(90)
        phi = np.deg2rad(0)
    if axis is "y":
        theta = np.deg2rad(90)
        phi = np.deg2rad(90)
    if axis is "z":
        theta = np.deg2rad(0)
        phi = np.deg2rad(0)
    if not rho:
        rho = curr_pos_dict['rho']
    start = rho

    stop = 0.01*rho
    sweep_arr = np.zeros(sweep_steps)
    sweep_arr[:-1] = np.logspace(np.log10(start), np.log10(stop), num=sweep_steps-1)
    aps_magnet.check_precision = 0.00001
    magnetlogic._checktime = 1.5

    b_arr = []
    for idx, rho_val in enumerate(sweep_arr):
        theta = np.pi-theta
        phi = np.pi+phi
        phi = phi if phi<2*np.pi else phi-2*np.pi
        move_dict =  {'rho':0, 'theta':np.rad2deg(theta), 'phi': np.rad2deg(phi)}
        print(move_dict)
        b_arr.append(move_dict)
        move_dict =  {'rho':rho_val, 'theta':np.rad2deg(theta), 'phi': np.rad2deg(phi)}
        print(move_dict)
        b_arr.append(move_dict)
    return b_arr

### Moving magnet along the ramp down directions

In [6]:
b_arr = ramp_along_set_direction(sweep_steps=10, theta=None, phi=None, rho=None)
# b_arr = ramp_along_axes(sweep_steps=10, axis="y", rho=50e-3)

odmrlogic.stop_jupyter_meas = False
for move_dict in tqdm(b_arr):    
    check_start_time = time.time()
    magnetlogic._magnet_device.move_abs(move_dict)
    
    while magnetlogic._check_is_moving():
        if (time.time()-check_start_time)>600:
            print('Magnet check time out!')
            break
        time.sleep(magnetlogic._checktime)
    
    if odmrlogic.stop_jupyter_meas:
        print('Measurement force stopped!')
        break

  0%|                                                                                          | 0/20 [00:00<?, ?it/s]

Current position {'rho': 0.06969770440982974, 'theta': 58.90113983754255, 'phi': 177.0225335355159} 


{'rho': 0, 'theta': 121.09886016245746, 'phi': 357.0225335355159}
{'rho': 0.06969770440982975, 'theta': 121.09886016245746, 'phi': 357.0225335355159}
{'rho': 0, 'theta': 58.90113983754254, 'phi': 177.02253353551583}
{'rho': 0.0391938994605489, 'theta': 58.90113983754254, 'phi': 177.02253353551583}
{'rho': 0, 'theta': 121.09886016245746, 'phi': 357.02253353551583}
{'rho': 0.022040349362022378, 'theta': 121.09886016245746, 'phi': 357.02253353551583}
{'rho': 0, 'theta': 58.90113983754254, 'phi': 177.02253353551583}
{'rho': 0.012394199267897928, 'theta': 58.90113983754254, 'phi': 177.02253353551583}
{'rho': 0, 'theta': 121.09886016245746, 'phi': 357.02253353551583}
{'rho': 0.006969770440982975, 'theta': 121.09886016245746, 'phi': 357.02253353551583}
{'rho': 0, 'theta': 58.90113983754254, 'phi': 177.02253353551583}
{'rho': 0.00391938994605489, 'theta': 58.90113983754254, 'phi': 177.0225335

100%|#################################################################################| 20/20 [16:12<00:00, 48.61s/it]
